In [ ]:
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
# Modules for data manipulation
import numpy as np
import pandas as pd
import re

# Modules for visualization
import matplotlib.pyplot as plt
import seaborn as sb

# Tools for preprocessing input data
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Tools for creating ngrams and vectorizing input data
from gensim.models import Word2Vec, Phrases

# Tools for building a model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

# Tools for assessing the quality of model prediction
from sklearn.metrics import accuracy_score, confusion_matrix

import os
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import pandas as pd
import os
import io
import numpy as np
import zipfile
import sys
import datetime
from sklearn.model_selection import train_test_split
import requests
import shutil
import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs



MAX_NB_WORDS = 100000
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

from subprocess import check_output


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def load_dataset(url: str, sep: str = ',', decode: str = 'utf-8', keep_default_na: bool = True, error_bad_lines: bool = True):
    def requests_csv(url: str, decode: str):
        return (requests.get(url).content).decode(decode)
    
    return pd.read_csv(io.StringIO(requests_csv(url, decode)),
                       sep=sep,
                       keep_default_na=keep_default_na,
                       error_bad_lines=error_bad_lines
                       )
url_dataset = "https://media.githubusercontent.com/media/Y4rd13/datasets/main/upwork_dataset.csv" 

from sklearn.model_selection import train_test_split

df =  load_dataset(url_dataset, decode="cp1252", keep_default_na=True, error_bad_lines=True)
#train_df = train_df.sample(2000)
df['jobText'] = df['jobText'].astype(str) 
df['approved'] = df['approved'].astype(int)
train_df, test_df = train_test_split(df, test_size = 0.2, random_state=42)

In [ ]:

#load embeddings
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('/content/drive/MyDrive/Andres Giovanna FRL BERT /wiki.simple.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...


FileNotFoundError: ignored

In [ ]:
print("num train: ", train_df.shape[0])
print("num test: ", test_df.shape[0])

In [ ]:
train_labels = train_df['approved'].values 
test_labels = test_df['approved'].values

In [ ]:
import seaborn as sns
y_train = np.asarray(train_labels).astype('float32').reshape((-1,1))
y_test = np.asarray(test_labels).astype('float32').reshape((-1,1))


#visualize word distribution
train_df['doc_len'] = train_df['jobText'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
sns.distplot(train_df['doc_len'], hist=True, kde=True, color='b', label='doc len')
plt.axvline(x=max_seq_len, color='k', linestyle='--', label='max len')
plt.title('comment length'); plt.legend()
plt.show()

In [ ]:
raw_docs_train = train_df['jobText'].tolist()
raw_docs_test = test_df['jobText'].tolist() 
num_classes = 2

print("pre-processing train data...")
processed_docs_train = []
for doc in tqdm(raw_docs_train):
    tokens = tokenizer.tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_train.append(" ".join(filtered))
#end for

processed_docs_test = []
for doc in tqdm(raw_docs_test):
    tokens = tokenizer.tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_test.append(" ".join(filtered))
#end for

print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  #leaky
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

#pad sequences
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

In [ ]:
#training params
batch_size =  8
num_epochs = 20

#model parameters
num_filters = 8
embed_dim = 300 
weight_decay = 1e-4

In [ ]:
#embedding matrix
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

In [ ]:
print("sample words not found: ", np.random.choice(words_not_found, 10))

In [ ]:
#CNN architecture
print("training CNN ...")
model = Sequential()
model.add(Embedding(nb_words, embed_dim,
          weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(2, activation='tanh'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

In [ ]:
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [ ]:
#model training
hist = model.fit(word_seq_train, y_train, batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)

# ####################################"

In [1]:
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
# Modules for data manipulation
import numpy as np
import pandas as pd
import re

# Modules for visualization
import matplotlib.pyplot as plt
import seaborn as sb

# Tools for preprocessing input data
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Tools for creating ngrams and vectorizing input data
from gensim.models import Word2Vec, Phrases

# Tools for building a model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

# Tools for assessing the quality of model prediction
from sklearn.metrics import accuracy_score, confusion_matrix

import os
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import pandas as pd
import os
import io
import numpy as np
import zipfile
import sys
import datetime
from sklearn.model_selection import train_test_split
import requests
import shutil
import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs



MAX_NB_WORDS = 100000
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

from subprocess import check_output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re


In [3]:
def load_dataset(url: str, sep: str = ',', decode: str = 'utf-8', keep_default_na: bool = True, error_bad_lines: bool = True):
    def requests_csv(url: str, decode: str):
        return (requests.get(url).content).decode(decode)
    
    return pd.read_csv(io.StringIO(requests_csv(url, decode)),
                       sep=sep,
                       keep_default_na=keep_default_na,
                       error_bad_lines=error_bad_lines
                       )
url_dataset = "https://media.githubusercontent.com/media/Y4rd13/datasets/main/upwork_dataset.csv" 

from sklearn.model_selection import train_test_split

data =  load_dataset(url_dataset, decode="cp1252", keep_default_na=True, error_bad_lines=True)
#train_df = train_df.sample(2000)
data['jobText'] = data['jobText'].astype(str) 
data['approved'] = data['approved'].astype(str)
#train_df, test_df = train_test_split(train_df, test_size = 0.2, random_state=42)

In [4]:

def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

data['jobText'] = data['jobText'].map(lambda com : clean_text(com))

In [5]:

RE_PATTERNS = {
    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*', 
             'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck','fuk','fk'
        ],
    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$','[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ],
    ' asshole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole'
        ],
    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h'
        ],
    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],
    ' trans gender':
        [
            'transgender'
        ],
    ' gay ':
        [
            'gay' , 'g4y'
        ],
    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],
    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k','dik'
        ],
    ' suck ':
        [
             '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'suck', '5uck', 's u c k'
        ],
    ' cunt ':
        [
            'cunt', 'c u n t'
        ],
    ' bullshit ':
        [
            'bullsh\*t', 'bull\$hit'
        ],
    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)',  'i d i o t'
                                                                                     
        ],
    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],
    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t'
        ],
    ' shit hole ':
        [
            'shythole'
        ],
    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],
    ' rape ':
        [
            ' raped'
        ],
    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],
    ' ass head':
        [
            'butthead'
        ],
    ' sex ':
        [
             's3x'
        ],
    ' nigger ':
        [
            'nigger', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],
    ' nigga ':
        [
            'niga', 'ni[g]+a', ' nigg[a]+'
        ],
    ' shut the fuck up':
        [
            'stfu'
        ],
    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses'
        ],
    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],
    ' motherfucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker',
        ],
    ' whore ':
        [
             'w h o r e'
        ],
}


for target,patterns in RE_PATTERNS.items():
  for pat in patterns:
    data['jobText'] = data['jobText'].map(lambda x : re.sub(pat,target,x))

In [6]:
data['jobText'] = data['jobText'].apply(lambda x: x.lower())
# removing special chars
data['jobText'] = data['jobText'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['jobText'] = data['jobText'].str.replace('rt','')
data.head()

,title,jobRegion,jobText,jobExperienceLevel,jobType,clientLocation,clientRating,approved
0,Web Design & Development Team (on-going work),Worldwide,we need a team that is capable of working with...,Expert,Complex project,United States,5.0,1
1,Website development,Worldwide,we are looking for someone to build a website ...,Expert,One-time project,United States,5.0,1
2,Auction Website to sell Household items,Worldwide,i am looking to hire a agency or freelancers w...,Expert,Complex project,United States,0.0,0
3,App development (Android & iOS),Worldwide,i am looking for someone that can make a compl...,Intermediate,One-time project,Sweden,4.0,0
4,Mobile App Development (Media playback app),Worldwide,what the deliverable is an audio player mobile...,Intermediate,One-time project,United States,5.0,0


In [7]:
max_fatures = 5000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['jobText'].values)
X = tokenizer.texts_to_sequences(data['jobText'].values)
X = pad_sequences(X)

Y = pd.get_dummies(data['approved']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3507, 885) (3507, 2)
(877, 885) (877, 2)


In [8]:
from keras.optimizers import Adam

In [9]:
embed_dim = 128
lstm_out = 100

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())



"""model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(256)))
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))


adam_opt = Adam(learning_rate=0.01)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])"""

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 885, 128)          640000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 885, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               91600     
_________________________________________________________________
dense (Dense)                (None, 2)                 202       
Total params: 731,802
Trainable params: 731,802
Non-trainable params: 0
_________________________________________________________________
None


'model = Sequential()\nmodel.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))\nmodel.add(Bidirectional(LSTM(256, return_sequences=True)))\nmodel.add(Dropout(0.2))\nmodel.add(Bidirectional(LSTM(256)))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(128, activation="relu"))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(64, activation="relu"))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(2, activation=\'sigmoid\'))\n\n\nadam_opt = Adam(learning_rate=0.01)\n\nmodel.compile(loss=\'binary_crossentropy\', optimizer=\'adam\', metrics = [\'accuracy\'])'

In [10]:
"""early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]"""

"early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)\ncallbacks_list = [early_stopping]"

In [11]:
"""hist = model.fit( = 32
model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list, shuffle=True, verbose=1)"""

'hist = model.fit( = 32\nmodel.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list, shuffle=True, verbose=1)'

In [ ]:
batch_size = 3
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 1)

Epoch 1/5
1169/1169 [==============================] - 1397s 1s/step - loss: 0.3817 - accuracy: 0.8295
Epoch 2/5
 581/1169 [=============>................] - ETA: 11:43 - loss: 0.2267 - accuracy: 0.9168

In [ ]:
Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))